In [ ]:
!pip install langchain langchain-openai langgraph langsmith python-dotenv --quiet

In [1]:


from dotenv import load_dotenv
import os

load_dotenv()

print("OPENAI_API_KEY loaded:", bool(os.getenv("OPENAI_API_KEY")))
print("LANGSMITH_API_KEY loaded:", bool(os.getenv("LANGSMITH_API_KEY")))
print("LANGSMITH_TRACING:", os.getenv("LANGSMITH_TRACING"))
print("Project:", os.getenv("LANGSMITH_PROJECT"))

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

from typing import List, TypedDict
from langgraph.graph import StateGraph, END

# ---- State Definitions ----
class Todo(TypedDict):
    task: str
    status: str

class AgentState(TypedDict):
    user_task: str
    todos: List[Todo]

# ---- Tool: write_todos ----
def write_todos(steps: List[str]) -> List[Todo]:
    return [{"task": step} for step in steps]

# ---- Planner Node ----
def planner_node(state: AgentState):
    prompt = (
        "You are a planning agent.\n"
        "Break the following complex task into 5–7 clear logical steps.\n"
        "Each step should be on a new line.\n\n"
        f"Task: {state['user_task']}\n\n"
        "Return ONLY the steps. Do NOT number them."
    )

    response = llm.invoke([prompt])

    steps = [
        line.strip()
        for line in response.content.split("\n")
        if line.strip()
    ]

    todos = write_todos(steps)
    return {"todos": todos}

graph = StateGraph(AgentState)

graph.add_node("planner", planner_node)
graph.set_entry_point("planner")
graph.add_edge("planner", END)

app = graph.compile()

test_tasks = [
    "Research the current market for solid-state batteries in 2024, identify the top 3 manufacturers, and summarize their latest technological breakthroughs." , 

"Find the latest NASA mission updates for Mars exploration, list the active rovers, and explain the primary goal of each." ,

"Analyze the impact of remote work on commercial real estate in New York City over the last 2 years and provide a summary of trends." ,

"Create a comprehensive study guide for a student learning Python for data science, covering libraries like Pandas, NumPy, and Scikit-learn." ,

"Investigate the supply chain for rare earth minerals used in EV production and identify the top three countries controlling the supply."
]

results = []

for idx, task in enumerate(test_tasks, 1):
    print(f"\n==============================")
    print(f"Task {idx}: {task}")
    print("==============================")

    output = app.invoke({
        "user_task": task,
        "todos": []
    })

    for i, todo in enumerate(output["todos"], 1):
        print(f"{i}. {todo['task']}")


OPENAI_API_KEY loaded: True
LANGSMITH_API_KEY loaded: True
LANGSMITH_TRACING: true
Project: AI_Autonomous Cognitive Engine

Task 1: Research the current market for solid-state batteries in 2024, identify the top 3 manufacturers, and summarize their latest technological breakthroughs.
1. Define the scope of the research, including key aspects of the solid-state battery market to focus on, such as applications, trends, and geographical regions.
2. Gather data from reliable sources, including industry reports, market analysis, and news articles related to solid-state batteries in 2024.
3. Analyze the collected data to identify the top 3 manufacturers based on market share, technological advancements, and product offerings.
4. Investigate each of the top manufacturers to find information on their latest technological breakthroughs in solid-state battery development.
5. Summarize the findings for each manufacturer, highlighting their innovations, competitive advantages, and potential impact